In [ ]:
# PYthon version 3.* 에서만 지원함
# 외부 모듈 socket,pymysql  설치해야함 
# 예외처리가 매우 미흡함으로 나중에 개선할 것 !!!!! 중요 !!!!!
import TcpNet
try:
    import pymysql
except ImportError:
    print('not pymysql')

def package_V(s): # 문자열 쌓아주는 함수
    return '\''+str(s)+'\''
    
DB_Host= '203.234.62.115' # DB 주소
DB_User='root' # 접속할 아이디 
DB_password ='1234' # 접속할 아이디의 비밀번호
DB_name = 'MetadataRegistry' # 접속할 DB의 이름
Specific_table_name = 'specific_metadata' # specific_metadata테이블 이름
charset = 'utf8' # 형식

table_name = '' #  데이터넣을 테이블 이름 변수 선언
item_id = ''
Device_Register_table_name = 'device_register' # 디바이스 리스트들은 테이블 이름

conn = pymysql.connect(host=DB_Host,user=DB_User, password=DB_password,db=DB_name, charset=charset) # DB연결 나중에 예외처리 해줄것
curs = conn.cursor() # DB 커서
print('DB connected.....') 

HOST='203.234.62.109' # 서버열 주소 나중에 자동으로 할당 될 수 잇도록 개선 할 것
PORT=11000 # 포트 는 10000이상으로 쓰고 겹치지 않는지 확인하며 할당 할 것

Tcp = TcpNet.TcpNet() # 객체선언
# 아직 쓰레드 상태가 아님
while True:
    print('waiting...')
    Tcp.Accept(HOST,PORT) # 클라이언트 접속 대기
    try:
        receive_id=Tcp.ReceiveStr(); # 클라이언트의 system.id 가 DB에 있는지 확인 후 tcble_name가져오기
        curs.execute("SELECT system_id,table_name, item_id FROM "+DB_name+"."+Device_Register_table_name +" WHERE system_id = " +
                     package_V(receive_id)+";")
        result = curs.fetchone();
        if(result[0] is not None):
            table_name=result[1]
            item_id=result[2]
            Tcp.SendStr('yes')
        else:
            Tcp.SendStr('no')
            break;
    except Exception as e :
        Tcp.SendStr('no')
        print('error :',e)
        break;
        
    print('Connected........')
    
    while True:
        ack= Tcp.ReceiveStr() # ACK MASSAGE 
        if (ack=='send'):
            Tcp.SendStr('con')
            
            
        receive_data=Tcp.ReceiveStr() # 내용 받기
        print("receive_data:"+receive_data) # 받은 내용 확인
        input_data=receive_data.split('!') # 포멧에 맞게 짜르기
        
        num=curs.execute("SELECT metadata_value FROM "+DB_name+'.'+Specific_table_name+" WHERE item_id = "+package_V(item_id)+" AND (metadata_key like "+package_V('sensor-%')+");") # 특정 table의 칼럼값을 가져옴
        DB_column= curs.fetchall() # %주의%  ((data,)....) 이중 튜플 형식으로 받아옴 가져오는 칼럼에 넣을 이름 중 timestammp는 없음
        
        DB_sql= 'INSERT INTO '+ table_name+' ( timestamp,' # 데이터 넣을 테이블
        for i in DB_column:
            for j in i:
                DB_sql= DB_sql+j+',' # 데이터 넣을 칼럼들 넣어주기
        DB_sql= DB_sql[:len(DB_sql)-1]+') VALUES ('

        a= package_V(input_data[0]) # 처음 받은 것은 timestamp(yyyy-mm-dd hh:mm;ss) 형식임으로 그 부분은 적출해 놓기
        input_data=input_data[1:] 
        data_list=[] # 데이터의 헤더값과 밸류값 분류할 컨테이너
        for i in input_data:
            data_list.append(i.split(':')) # 분류 작업
        data_dict= dict(data_list) # 딕셔너리 형태로 바꿔주기
        
        input_data=[] # 넣을 데이터 담을 컨테이너
        for i in DB_column:
            for j in i:
                if j in data_dict:
                    input_data.append(data_dict[j]) # 칼럼들 서칭후 칼럼 순에 맞게 데이터 넣기
                else:
                    input_data.append('NULL')
                
        input_data.insert(0,(a)) # 앞쪽에 timestamp넣기
        s=",".join(input_data) # 데이터를 넣는데 ,로 구분해서 넣기
        DB_sql= DB_sql+s+');' # 마감 작업
        curs.execute(DB_sql)
        conn.commit()
        ack=None
        if receive_data == 'exit' :
            print('socket end')
            break